## 크롤링 코드

Statiz 사이트의 경기 결과를 크롤링

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
import time

# 👉 전체 데이터 저장 리스트 (타격 + 투구)
all_team_totals = []

# 👉 열 이름 정의 (미리)
columns = (
    ['GameID', 'Team'] +
    ['PA', 'AB', 'R', 'H', 'HR', 'RBI', 'BB', 'HBP', 'SO',
     'GO', 'FO', 'NP', 'GDP', 'LOB', 'AVG', 'OPS', 'LI_b', 'WPA_b', 'RE24_b'] +
    ['IP', 'TBF', 'H_p', 'R_p', 'ER', 'BB_p', 'HBP_p', 'K', 'HR_p',
     'GO-FO', 'NP-S', 'IR-IS', 'GSC', 'ERA', 'WHIP', 'LI_p', 'WPA_p', 'RE24_p']
)

# 👉 CSV 파일명
csv_file = '2025_05_28_total.csv'

# 👉 기존 파일이 있으면 불러오기
if os.path.exists(csv_file):
    existing_df = pd.read_csv(csv_file)
    print(f"기존 {csv_file} 불러옴, {len(existing_df)}개의 행이 존재함")
else:
    existing_df = pd.DataFrame(columns=columns)

# 👉 날짜 반복
for num in range(251, 291):
    time.sleep(60)
    game_id = f"2025{num:04d}"

    try:
        url = f'https://statiz.sporki.com/schedule/?m=boxscore&s_no={game_id}'
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36'}
        res = requests.get(url, headers=headers)
        res.raise_for_status()
        soup = BeautifulSoup(res.text, 'html.parser')

        # 👉 타격 기록 저장용 임시 딕셔너리 (team_name → 타격 데이터)
        batting_dict = {}

        # ✅ 타격 기록
        batting_sections = soup.select('div.box_type_boared .item_box')
        for section in batting_sections:
            head = section.select_one('.box_head')
            if head and '타격기록' in head.text:
                team_name_raw = head.text.replace('타격기록', '').strip()
                team_name = re.sub(r'[()]', '', team_name_raw)
                total_row = section.select_one('tr.total')
                if total_row:
                    values = [td.text.strip() for td in total_row.find_all('td') if not td.has_attr('colspan')]
                    if len(values) == 19:
                        batting_dict[team_name] = values

        # ✅ 투구 기록
        pitching_tables = soup.select('div.box_type_boared .box_head')
        for head in pitching_tables:
            if '투구기록' in head.text:
                team_name_raw = head.text.replace('투구기록', '').strip()
                team_name = re.sub(r'[()]', '', team_name_raw)
                total_row = head.find_next('tr', class_='total')
                tds = total_row.find_all('td')

                if len(tds) >= 19:
                    pitching_values = [td.text.strip() for td in tds[1:]]
                    if team_name in batting_dict:
                        batting_values = batting_dict[team_name]
                        # 날짜 정보 추출
                        score_box = soup.select_one('div.callout_box div.score .txt')
                        if score_box:
                            full_text = score_box.text.strip()  # 예: "고척, 04-03"
                            if ',' in full_text:
                                date_part = full_text.split(',')[-1].strip()  # "04-03"
                        all_team_totals.append([date_part, team_name] + batting_values + pitching_values)

    except Exception as e:
        print(f"❌ 오류 발생: {game_id} - {e}")

        # 👉 지금까지 수집된 데이터 저장 후 종료
        if all_team_totals:
            new_df = pd.DataFrame(all_team_totals, columns=columns)
            combined_df = pd.concat([existing_df, new_df], ignore_index=True)
            combined_df.to_csv(csv_file, index=False, encoding='utf-8-sig')
            print(f"✅ 오류 직전까지 {len(all_team_totals)}건 저장 완료")
        else:
            print("⛔ 저장할 데이터 없음")
        break

# 👉 전체 루프 완료 후 저장
if all_team_totals:
    new_df = pd.DataFrame(all_team_totals, columns=columns)
    combined_df = pd.concat([existing_df, new_df], ignore_index=True)
    combined_df.to_csv(csv_file, index=False, encoding='utf-8-sig')
    print(f"🎉 전체 완료! 총 {len(combined_df)}건 저장됨. {num}까지 완료 다음 번 {num+1}부터 시작")


🎉 전체 완료! 총 78건 저장됨. 290까지 완료 다음 번 291부터 시작
